## Outlier Detection

Trovare i valori anomali nei dati è un processo molto delicato perchè presuppone la conoscenza del fenomeno che si và ad osservare.
<br>
**Che cos'è un outlier?** In maniera più semplice possibile *un outlier è un'osservazione che differisce significativamente dalle altre della stessa variabile*.
Esistono molti tipi di outlier ad esempio:
* molte più visite ad un sito per un breve periodo di tempo: **outlier additivo**
* i server del sito vanno offline e non ricevete visite: **cambiamento temporale**
<br>

Nelle serie storiche questa differenza deve essere ancora più marcata perchè deve differire dai pattern comportamentali della serie, utilizzando sempre Airpassenger, abbiamo già visto che ha un trend crescente con componente stagionale ad effetto moltiplicativo.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
airpassengers = pd.read_csv("airpassenger.csv")
airpassengers.columns = ["Time","Passengers"]
#airpassengers.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers", xlabel="Time", ylabel="Passengers")
plt.show()

In questo caso non possiamo affermare che l'osservazione 140 è un outlier solo perchè è nettamente maggiore della 15, perchè segue il trend della serie. Allo stesso modo **i picchi non possono essere considerati outlier perchè effetto della stagionalità.**
<br>
Introduciamo artificialmente degli outlier in airpassenger:
* creare un vettore di 10 interi in maniera completamente casuale
* sostituire nelle 10 posizioni un valore molto alto

In [ ]:
idx = np.random.randint(0, airpassengers.shape[0], 10)
airpassengers.iloc[idx,1] = 1000

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), airpassengers["Passengers"].values, color='tab:blue')
plt.gca().set(title="Airpassengers with outlier", xlabel="Time", ylabel="Passengers")
plt.show()

In questo notebook vedremo **alcuni** metodi per trattare i valori anomali nelle serie storiche univariate.
<br>
L'analisi statistica delle serie storiche fornisce già molte tecniche, se si aggiungono quelle di ML/DL questo notebook potrebbe diventare infinito.
<br>
***N.B. In coda troverete alcuni link che potrebbero essere interessanti***

### Decomposizione della serie e analisi dei residui
La prima tecnica è scomporre la serie nelle sue componenti principali e studiarne i residui.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
seas = seasonal_decompose(airpassengers.Passengers, model = 'multiplicative', freq=12).plot()

Possiamo subito notare che, guardando i residui si possono facilmente individuare gli outlier.
<br>
Ora andrebbero condotti dei test, solitamente il t-student test, per verificare se sono davvero solo questi gli outlier della serie, o se queste valli non sono outlier. Questa tecnica è abbastanza semplice ma anche molto rigida quindi se la  serie deriva da un fenomeno molto erratico può indurre in valutazioni non corrette. 

### Outlier detection utilizzando la distribuzione Gaussiana
Questa tecnica è molto utilizzata ed insegnata ma presuppone che i dati si distribuiscano come una gaussiana (indipendente, identicamente distribuita), cosa che non succede spesso nella realtà, quindi la tralascerò ma nei link utili trovate un articolo (con codice) per testare l'approccio.

### Support Vector Machine SVM
Le SVM sono solitamente associate alla classificazione, in particolare alla classificazione binaria ma possono essere utilizzate per trovare gli outlier, in particolare sono state utilizzate nella **novelty detection**.
<br>
Il loro funzionamento in questo campo è semplice, trovano una funzione che positiva nelle regioni con alta densità di punti e negativa dove c'è bassa densità, quindi anomalie.
<br>
Prima di applicare l'algoritmo applichiamo una traformazione scalando i valori tra [-1,1].

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler((-1,1))
scaled_data = scaler.fit_transform(airpassengers[["Passengers"]])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(range(airpassengers.shape[0]), scaled_data, color='tab:blue')
plt.gca().set(title="Airpassengers Scaled", xlabel="Time", ylabel="Passengers - Scaled")
plt.show()

Importiamo OneClassSVM e impostiamo due parametri:
* *nu* come la percentuale di outlier da cercare, io ne ho inseriti 10 su circa 140 osservazioni e quindi inserisco il 10%
* gamma è il coefficiente della funzione Kernel utilizzata dalla SVM, in questo caso avendo una serie univariata sarebbe:
    * 1 = 1/n_features col metodo *auto*
    * 1/(n_features * varianza(serie)) col metodo *scale*
<br>

Questo metodo crea un vettore che è:
* 1 se il dato è conforme alla distribuzione osservata
* -1 valore anomalo

In [ ]:
from sklearn.svm import OneClassSVM
model = OneClassSVM(nu=0.1, kernel="rbf", gamma="scale")
model.fit(scaled_data)
df = pd.DataFrame(scaled_data, columns=["Scaled Data"])
df["anomalySVM"] = pd.Series(model.predict(scaled_data))

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
a = df.loc[df['anomalySVM'] == -1, ['Scaled Data']] #anomaly

ax.plot(range(df.shape[0]), df['Scaled Data'], color='blue')
ax.scatter(a.index,a['Scaled Data'], color='red')
plt.show();

L'algoritmo trova più valori anomali di quanti ne avevamo indotti, normale avendo sovrastimato la percentuale. Questo algoritmo usa una funzione che crea bordi molto regolari per dividere racchiedere gli outlier, potrebbe essere un problema.
<br>
**N.B. Nei link utili anche uno sulla comparazione di diverse tecniche dove la rappresentazione grafica aiuta a capire meglio il problema**

### Isolation Forest
Isolation Forest calssifica le anomalie semplicemente utilizzando il fatto che queste sono estremamente differenti dagli altri valori, quindi costruisce degli alberi di classificazione e se il path di classifcazione di questi punti è significativamente più breve degli altri sono anomalie.
<br>
**[ATTENZIONE!!! Spiegazione fortemente semplicistica]**

<br>

Utilizziamo sempre il 10% come soglia, questa volta il parametro è *contamination* nella documentazione troviamo i bound del parametro, valore massimo 0.5, se supera il 50% non è più un'anomalia. Il parametro *behavior* è deprecato nella nuova versione 0.22 si scikit-learn ma verrà rimosso solo nella 0.24, quindi per non avere warning lo impostiamo a new come chiede la documentazione. **[leggere la documetazione fa risparmiare molto tempo]**
<br>
Anche in questo caso viene restituito i un vettore di -1 *anomalia*, 1 *valore standard*.

In [ ]:
from sklearn.ensemble import IsolationForest

isoF = IsolationForest(contamination=0.1, behaviour="new")
isoF.fit(scaled_data)
df["anomalyIsoF"] = pd.Series(isoF.predict(scaled_data))

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
a = df.loc[df['anomalyIsoF'] == -1, ['Scaled Data']]

ax.plot(range(df.shape[0]), df['Scaled Data'], color='blue')
ax.scatter(a.index,a['Scaled Data'], color='red')
plt.show();

Anche in questo caso trova più valori di quelli indotti ma almeno le osservazioni artificiali vengono segnalate.
<br>
Questa volta i nostri esperimenti non hanno avuto grande successo, in parte è dovuto alle poche osservazioni del dataset che si confà più a tecniche di statistica classica come lo studio dei residui piuttosto che al ML.
<br>
**N.B. ad ogni rilancio dell'intero notebook i vostri valori anomali cambierianno quindi potreste trovare commenti disallineati con il risultato del vostro esperimento**

### Appendice: Novelty vs Anomaly Detection
La differenza principale tra le due analisi è:
* in Anomaly Detection viene passato in fase di train un dataset contenente anche outlier e il modello li classifica come tali
* in Novelty Detection il dataset di train è privo di outlier e quindi in fase di test trova i valori che si discostano da qunto osservato prima

<br>

### Link Utili
* [Comparing anomaly detection algorithms for outlier detection on toy datasets](https://scikit-learn.org/stable/auto_examples/plot_anomaly_comparison.html#sphx-glr-auto-examples-plot-anomaly-comparison-py)
* [Time Series Anomaly Detection Algorithms](https://blog.statsbot.co/time-series-anomaly-detection-algorithms-1cef5519aef2)
* [Outlier detection with time-series data mining](https://www.datasciencecentral.com/profiles/blogs/outlier-detection-with-time-series-data-mining)
* [Time Series of Price Anomaly Detection](https://towardsdatascience.com/time-series-of-price-anomaly-detection-13586cd5ff46)